In [13]:
import astropy.units as units
import astropy.constants as constants
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import requests
import re
import subprocess
import urllib.request
# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"
a, b, c, d, e, f, g, h, i, j, k, l, m = sym.symbols('a b c d e f g h i j k l m')
n, o, p, q, r, s, t, u, v, w, x, y, z = sym.symbols('n o p q r s t u v w x y z')
symbol_list = (a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v,
w, x, y, z)
A, B, C, D, E, F, G, H, I, J, K, L, M = sym.symbols('A B C D E F G H I J K L M')
N, O, P, Q, R, S, T, U, V, W, X, Y, Z = sym.symbols('N O P Q R S T U V W X Y Z')
symbol_list = (A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, 
               V, W, X, Y, Z)
#%matplotlib notebook #incompatible with mpmath

This function queries the International Atomic Energy Agency's Nuclear Data Services (https://nds.iaea.org/).

API: https://nds.iaea.org/relnsd/vcharthtml/api_v0_guide.html 

Example Jupyter Notebook for automating data downloads: https://www-nds.iaea.org/relnsd/vcharthtml/api_v0_notebook.html  

In [2]:
def lc_read_csv(url):
    '''
    Query the livechart service and return a pandas dataframe. 
    format: lc_read_csv(livechart + "fields=decay_rads&nuclides=" + "16O" + "&rad_types=bm")
    '''
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

 This function takes a dataframe of parent nuclides and returns a list of dataframes of daughter nuclides

In [3]:
def get_daughter_df(parent_df):
    '''
    This function takes a dataframe of parent nuclides and returns a 
    list of dataframes of daughter nuclides
    '''
    daughter_nucleus= [str(int(parent_df['d_z'][row]) + int(parent_df['d_n'][row])) + 
                    ''.join(parent_df['d_symbol'][row]) for row in range(parent_df.shape[0])]
    daughter_df = [lc_read_csv(livechart + "fields=decay_rads&nuclides=" + daughter_nucleus[row].lower() 
                    + "&rad_types=bm") for row in range(parent_df.shape[0])]
    daughter_df = pd.concat(daughter_df)
    return daughter_df


It appears that when queried for stable nuclides, livechart return a 0x1 dataframe with only the number 0

In [4]:
#It appears that when queried for stable nuclides, livechart return a 0x1 dataframe with only the number 0
lc_read_csv(livechart + "fields=decay_rads&nuclides=" + "16O" + "&rad_types=bm")

,0


In [5]:
df = lc_read_csv(livechart + "fields=decay_rads&nuclides=42ar&rad_types=bm")
df.iloc[ : , : 15]

,mean_energy,unc_me,intensity_beta,unc_ib,daughter_level_energy,max_energy,unc_me.1,log_ft,unc_lf,transition_type,anti_nu_mean_energy,unc_ame,p_z,p_n,p_symbol
0,233,16,100,0,0,599,6,9.32,0.17,1U,367,3.6,18,24,Ar


In [6]:
df.iloc[ : , 15: ]

,p_energy_shift,p_energy,unc_pe,jp,half_life,operator_hl,unc_hl,unit_hl,half_life_sec,unc_hls,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,,0,,0+,32.9,,11,Y,1.038223e+09,3.471262e+07,...,100,,599,6,19,23,K,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-05


In [7]:
float(df['mean_energy']) * float(df['intensity_beta']) / 100

233.0

In [8]:
daughter_nucleus = str(int(df['d_z']) + int(df['d_n'])) + ''.join(df['d_symbol'])
#make all letters lowercase 
daughter_nucleus.lower()

'42k'

In [9]:
daughter_df = lc_read_csv(livechart + "fields=decay_rads&nuclides=" + daughter_nucleus.lower() + "&rad_types=bm")
daughter_df.iloc[ : , : 15]

,mean_energy,unc_me,intensity_beta,unc_ib,daughter_level_energy,max_energy,unc_me.1,log_ft,unc_lf,transition_type,anti_nu_mean_energy,unc_ame,p_z,p_n,p_symbol
0,21.41,0.21,0.07,0.01,3445.4,79.80,0.70,4.9800,0.0700,A,58.60,0.60,19,23,K
1,415.41,0.2,0.05,0.01,2424.3,1100.90,0.40,9.0500,0.0900,1NU,686.70,0.60,19,23,K
2,702.95,0.2,0.34,0.03,1837.2,1688.00,0.40,9.9200,0.0400,1U,986.49,0.36,19,23,K
3,824.32,0.17,17.64,0.09,1524.6,2000.60,0.30,7.5501,0.0023,1NU,1177.16,0.25,19,23,K
4,1565.86,,81.90,0.09,0.0,3525.22,0.18,9.4768,0.0006,1U,1961.70,0.09,19,23,K


In [10]:
daughter_df.iloc[ : , 15: ]

,p_energy_shift,p_energy,unc_pe,jp,half_life,operator_hl,unc_hl,unit_hl,half_life_sec,unc_hls,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-05
1,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-05
2,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-05
3,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-05
4,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-05


In [11]:
daughter_df = get_daughter_df(df)

In [12]:
third_gen_df = get_daughter_df(daughter_df)
third_gen_df

,0


#Battery Object
We will make a list of dataframes. Each element of the list will correspond to a step in the decay chain.
Each simulated battery will be a battery object. Every battery object has a composition attribute. 
This composition will be a dictionary where the keys are the nuclei and the values are the moles of each 
nucleus. The class will have a chain decay function that calculates how many moles of each nucleus there are at a given time.

#Derivation of chain decay function. This function will calculate the moles of each nucleus after a given time ($\tau$). The are $m_{t}$ moles of the mother nucleus at time $t$ and $\delta_{\tau}$ moles of the daughter nucleus at a time $\tau$. There is a probability $P_{x}$ that a given nucleus $x$ with an age of $t$ and an e-folding time of $\lambda_{x}$ has not yet decayed.
\begin{equation}
\begin{split}
\Large{ m_t = m_0e^{\Large\frac{t}{-\lambda_{m}}} } \\
\Large{ P_{x} = 1-e^{\Large\frac{t_{x}}{-\lambda_{x}}} }\\
\Large{ \delta_{\tau} = \int_{0}^{\tau} \delta_{new}P_{\delta}dt }\\
\end{split}
\end{equation}
$\delta_{new}$ is the number of daughter nuclei that were created instantaneously at the time $t$. In these decay chains, the only source of new daughter nuclei is the decay of the mother nuclei so we have
\begin{equation}
\begin{split}
\Large{ \delta_{age} = -\frac{d_{m}}{dt} = \frac{m_0e^{\Large\frac{t}{-\lambda_{m}}}}{\lambda_{m} } }\\
\Large{ \delta_{\tau} = \int_{0}^{\tau} \frac{m_0e^{\Large\frac{t}{-\lambda_{m}}}}{\lambda_{m} }(1-e^{\Large\frac{t}{-\lambda_{\delta}}}) dt}\\
\Large{ \delta = \frac{m_0}{\lambda_{m}}\int_{0}^{\tau} e^{\Large\frac{t}{-\lambda_{m}}}(1-e^{\Large\frac{t}{-\lambda_{\delta}}}) dt}\\
\Large{ \delta_{\tau} = \frac{m_0}{\lambda_{m}}(\int_{0}^{\tau} e^{\Large\frac{t}{-\lambda_{m}}}dt - \int_{0}^{\tau} e^{\Large (\frac{t}{-\lambda_{\delta}} + \frac{t}{-\lambda_{m}})} dt) }\\
\end{split}
\end{equation}
for the purposes of evaluating the integrals we have $\delta = -\lambda_{\delta}, m = -\lambda_{m}$

https://www.wolframalpha.com/input?i=%5Cint_%7B0%7D%5E%7B%5Ctau%7D+e%5E%7B%5Cfrac%7Bt%7D%7Bm%7D%7D%281-e%5E%7B%5Cfrac%7Bt%7D%7B%5Cdelta%7D%7D%29+dt+
\begin{equation}
\begin{split}
\int_{0}^{\tau} e^{\frac{t}{m}}(1-e^{\frac{t}{\delta}}) dt = m(-\frac{\delta(e^{\tau(m^{-1} + \delta^{-1})})}{\delta + m} + e^\frac{\tau}{m} -1) \\
\end{split}
\end{equation}
Substituting and returning to our original notation we have
\begin{equation}
\begin{split}
\Large
\delta_{\tau} = m_0(-\frac{\lambda_{\delta}e^{\Large \tau(\lambda_{m}^{-1} + \lambda_{\delta}^{-1})}}{\lambda_{\delta} + \lambda_{m}} + e^{\Large\frac{\tau}{\lambda_{m}}} -1)\\
\end{split}
\end{equation}